# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter

In [65]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [58]:
import codecs
dvach = codecs.open( "lenta.txt", "r", "utf_8_sig" ).read() 

norm_news = normalize(news)

vocab_news = Counter(norm_news)

probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [59]:
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [60]:
unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence,2))
    threegrams_news.update(ngrammer(sentence))

In [61]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [70]:
matrix_news = lil_matrix((len(bigrams_news), 
                        len(unigrams_news)))

id2word_news_1 = list(unigrams_news)
word2id_news_1 = {word:i for i, word in enumerate(id2word_news_1)}

id2word_news_2 = list(bigrams_news)
word2id_news_2 = {word:i for i, word in enumerate(id2word_news_2)}


for ngram in threegrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + " " + word2
    matrix_news[word2id_news_2[bigram], word2id_news_1[word1]] =  (threegrams_news[ngram]/
                                                                     bigrams_news[bigram])
    
matrix_news = csc_matrix(matrix_news)

In [125]:
print(matrix_news)

  (np.int32(0), np.int32(0))	3.1006790487116676e-05
  (np.int32(1), np.int32(0))	0.2
  (np.int32(10), np.int32(0))	1.0
  (np.int32(22), np.int32(0))	0.0003056234718826406
  (np.int32(30), np.int32(0))	0.004424778761061947
  (np.int32(40), np.int32(0))	0.5
  (np.int32(54), np.int32(0))	0.125
  (np.int32(76), np.int32(0))	1.0
  (np.int32(82), np.int32(0))	0.0032258064516129032
  (np.int32(101), np.int32(0))	0.0014619883040935672
  (np.int32(122), np.int32(0))	0.3333333333333333
  (np.int32(149), np.int32(0))	1.0
  (np.int32(204), np.int32(0))	0.01818181818181818
  (np.int32(212), np.int32(0))	0.00038804811796662784
  (np.int32(228), np.int32(0))	0.008403361344537815
  (np.int32(243), np.int32(0))	0.03125
  (np.int32(259), np.int32(0))	1.0
  (np.int32(273), np.int32(0))	0.004975124378109453
  (np.int32(282), np.int32(0))	1.0
  (np.int32(288), np.int32(0))	0.004329004329004329
  (np.int32(303), np.int32(0))	1.0
  (np.int32(323), np.int32(0))	1.0
  (np.int32(389), np.int32(0))	1.0
  (np.int

In [83]:
import numpy as np

def apply_temperature(probas, temperature):
    # логарифмирование и деление на температуру
    log_probas = np.log(np.maximum(probas, 1e-10))  
    adjusted_log_probas = log_probas / temperature
    # чтобы получить честные вероятности, нужно применить софтмакс
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

In [155]:
def generate(matrix, id2word, word2id, n=100, start='<start> <start>', temperature = 0.1):
    text = []
    current_idx = word2id[start]
    for i in range(n):
        
        chosen_idx = np.random.choice(matrix.shape[1], p=apply_temperature(matrix[current_idx].toarray()[0], temperature=temperature))
        chosen = id2word[chosen_idx]
        text.append(chosen)

        if id2word[chosen_idx] == '<end>':
            chosen_idx = word2id['<start> <start>']
        current_idx = chosen_idx
        print(current_idx)
    return ' '.join(text)

In [156]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
<start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start> <start>


## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.